# 🚀 AlphaBot ML/DL Training - Google Colab (v2)

Notebook propre et corrigé (détection robuste des colonnes yfinance MultiIndex/suffixes) avec suivi/reprise, téléchargement de données robuste et fallbacks sûrs.

In [ ]:
# Ajustement du path pour que Colab trouve le module alphabot
import sys
sys.path.append('/content')
sys.path.append('/content/alphabot')

## 🔄 Suivi de Progression et Reprise Automatique

In [ ]:
# 🔄 Système de suivi et reprise automatique
import os, json
from datetime import datetime

base_path = '/content/drive/MyDrive/AlphaBot_ML_Training'
os.makedirs(base_path, exist_ok=True)
progress_file = f'{base_path}/progress_tracker.json'

default_progress = {
    'cell_1_setup': False,
    'cell_2_data_download': False,
    'cell_3_data_analysis': False,
    'cell_4_pattern_training': False,
    'cell_5_sentiment_training': False,
    'cell_6_rag_training': False,
    'cell_7_integration': False,
    'cell_8_testing': False,
    'cell_9_deployment': False,
    'last_cell_executed': None,
    'start_time': None,
    'last_update': None
}

try:
    with open(progress_file, 'r') as f:
        progress = json.load(f)
    print('📊 Suivi de progression chargé')
except Exception:
    progress = default_progress.copy()
    progress['start_time'] = datetime.now().isoformat()
    print('🆕 Nouveau suivi de progression initialisé')

def update_progress(cell_name):
    progress.setdefault(cell_name, False)
    progress[cell_name] = True
    progress['last_cell_executed'] = cell_name
    progress['last_update'] = datetime.now().isoformat()
    with open(progress_file, 'w') as f:
        json.dump(progress, f, indent=2)
    print(f'✅ Progression mise à jour: {cell_name}')

def check_progress():
    print('\n📋 État actuel de la progression:')
    print('=' * 50)
    completed = sum(1 for k, v in progress.items() if k.startswith('cell_') and isinstance(v, bool) and v)
    total = sum(1 for k in default_progress.keys() if k.startswith('cell_'))
    pct = (completed / total * 100) if total else 0.0
    print(f'📊 Progression: {completed}/{total} étapes complétées ({pct:.1f}%)')
    print(f'⏰ Démarré: {progress.get("start_time", "N/A")}')
    print(f'🔄 Dernière mise à jour: {progress.get("last_update", "N/A")}')
    print(f'📍 Dernière cellule: {progress.get("last_cell_executed", "Aucune")}')
    print('=' * 50)

check_progress()
print('\n💡 Instructions:')
print('1. Exécutez cette cellule pour voir l\'état d\'avancement')
print('2. Chaque cellule mettra à jour automatiquement sa progression')
print('3. Si le processus s\'arrête, relancez simplement cette cellule')
print('4. Continuez avec la cellule suggérée')

## 1) Setup GPU/TPU et environnement Colab

In [ ]:
import sys, os, subprocess

def pip_install(pkgs):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + pkgs)

# Nettoyage des modules déjà importés
for m in [m for m in list(sys.modules) if m.startswith('transformers') or m.startswith('accelerate') or m.startswith('numpy') or m.startswith('tensorflow') or m.startswith('torch')]:
    del sys.modules[m]

# Réinstaller numpy pour compatibilité binaire
pip_install([
    '--upgrade',
    'numpy>=1.24.0,<1.27.0'
])

# Installer un set compatible (Option A)
pip_install([
    'transformers>=4.43,<4.47',
    'accelerate>=0.30,<0.34',
    'datasets>=2.18,<3.0',
    'safetensors>=0.4.3',
    'huggingface-hub>=0.23,<0.25'
])

os.environ['HF_HUB_DISABLE_TELEMETRY'] = '1'

# Importer et afficher les versions
import numpy
import transformers
print('NumPy version:', numpy.__version__)
print('Transformers version:', transformers.__version__)

import tensorflow as tf
import torch, logging
logging.basicConfig(level=logging.INFO)

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print('✅ TPU détectée et configurée')
except Exception:
    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            strategy = tf.distribute.MirroredStrategy()
            print(f'✅ {len(gpus)} GPU(s) détectée(s)')
        else:
            strategy = tf.distribute.get_strategy()
            print('⚠️ Aucun GPU/TPU détecté, utilisation du CPU')
    except Exception as e:
        strategy = tf.distribute.get_strategy()
        print(f'⚠️ Erreur de configuration GPU: {e}')

try:
    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    tf.keras.mixed_precision.set_global_policy(policy)
    print('✅ Mixed precision activée')
except Exception:
    print('⚠️ Mixed precision non disponible')

print('\n📊 Configuration:')
print(f'- TensorFlow: {tf.__version__}')
print(f'- PyTorch: {torch.__version__}')
print(f'- Strategy: {strategy}')
print(f"- GPUs disponibles: {tf.config.list_physical_devices('GPU')}")
if torch.cuda.is_available():
    print(f'- CUDA: {torch.version.cuda}')
    print(f'- GPU: {torch.cuda.get_device_name(0)}')

update_progress('cell_1_setup')

## 2) Montage Google Drive (résilient v2)

In [ ]:
print('🔧 Configuration Google Drive (résiliente v2)...')
from google.colab import drive
import os, shutil, time
MOUNT_POINT = '/content/drive'

def _safe_cleanup_mount_point(mp: str):
    try:
        if os.path.islink(mp): os.unlink(mp)
        if os.path.isdir(mp):
            for entry in os.listdir(mp):
                p = os.path.join(mp, entry)
                try:
                    if os.path.isfile(p) or os.path.islink(p): os.remove(p)
                    elif os.path.isdir(p): shutil.rmtree(p)
                except Exception: pass
        else:
            os.makedirs(mp, exist_ok=True)
    except Exception as e:
        print(f'⚠️ Nettoyage mount point: {e}')

def _force_unmount():
    try: drive.flush_and_unmount()
    except Exception: pass
    try:
        os.system('fusermount -u /content/drive 2>/dev/null || true')
        os.system('umount /content/drive 2>/dev/null || true')
    except Exception: pass

_force_unmount(); time.sleep(1)
_safe_cleanup_mount_point(MOUNT_POINT); time.sleep(0.5)
try:
    drive.mount(MOUNT_POINT, force_remount=True)
    print('✅ Drive monté (v2)')
except Exception as e:
    print(f'❌ drive.mount a échoué: {e}')
    if 'Mountpoint must not already contain files' in str(e):
        try:
            shutil.rmtree(MOUNT_POINT, ignore_errors=True)
            os.makedirs(MOUNT_POINT, exist_ok=True)
            drive.mount(MOUNT_POINT, force_remount=True)
            print('✅ Drive monté après recréation du dossier')
        except Exception as e2:
            print(f'⚠️ Impossible de recréer {MOUNT_POINT}: {e2}')
            raise

base_path = '/content/drive/MyDrive/AlphaBot_ML_Training'
for sub in ('data', 'models', 'checkpoints', 'logs', 'exports'):
    os.makedirs(f'{base_path}/{sub}', exist_ok=True)
print(f'📁 Répertoires prêts sous: {base_path}')
update_progress('cell_2_data_download')

## 4) Téléchargement des données (robuste + fallbacks + sauvegarde CSV locale pour preuve)

In [ ]:
import os, ssl, time, pickle
from datetime import datetime, timedelta
import pandas as pd
import yfinance as yf

os.environ['PYTHONHTTPSVERIFY'] = '0'
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

symbols = ['AAPL','GOOGL','MSFT','TSLA','AMZN','NVDA','META','NFLX','IBM','ORCL','INTC','AMD','QCOM','CRM','SAP','CSCO','JPM','BAC','V','MA','DIS','KO','PEP','NKE','XOM','SPY','QQQ','DIA','IWM']
print(f'📥 Téléchargement (yfinance) period=10y interval=1d pour {len(symbols)} tickers...')

def dl_yf(symbol):
    try:
        df = yf.download(symbol, period='10y', interval='1d', auto_adjust=True, progress=False)
        if df is not None and not df.empty:
            return df
    except Exception as e:
        print(f'⚠️ yfinance {symbol}: {e}')
    return None

def dl_pdr_yahoo(symbol):
    try:
        from pandas_datareader import data as pdr
        yf.pdr_override()
        end = datetime.now(); start = end - timedelta(days=365*10)
        df = pdr.get_data_yahoo(symbol, start=start, end=end)
        if df is not None and not df.empty:
            return df
    except Exception as e:
        print(f'⚠️ pdr-yahoo {symbol}: {e}')
    return None

def dl_pdr_stooq(symbol):
    try:
        from pandas_datareader import data as pdr
        end = datetime.now(); start = end - timedelta(days=365*10)
        df = pdr.DataReader(symbol, 'stooq', start, end)
        if df is not None and not df.empty:
            return df.sort_index()
    except Exception as e:
        print(f'⚠️ stooq {symbol}: {e}')
    return None

def safe_download(symbol):
    for fn in (dl_yf, dl_pdr_yahoo, dl_pdr_stooq):
        df = fn(symbol)
        if df is not None and not df.empty:
            return df
    return None

all_data = {}
csv_out = '/content/market_data_csv'
os.makedirs(csv_out, exist_ok=True)
for s in symbols:
    df = safe_download(s)
    if df is not None and not df.empty:
        all_data[s] = df
        df.to_csv(f'{csv_out}/{s}.csv')
        print(f'✅ {s}: {len(df)} lignes (CSV écrit)')
    else:
        print(f'❌ {s}: vide (après yfinance+pdr)')

print('Symbols téléchargés:', list(all_data.keys()))
for s, df in list(all_data.items())[:10]:
    print(s, 'rows=', len(df))

data_path = f'{base_path}/data/market_data.pkl'
os.makedirs(f'{base_path}/data', exist_ok=True)
with open(data_path, 'wb') as f:
    pickle.dump(all_data, f)
print(f'💾 Données sauvegardées: {data_path}')
update_progress('cell_4_pattern_training')

## 5) Entraînement Pattern Detector — CORRIGÉ (détection robuste yfinance MultiIndex/suffixes)

In [ ]:
import os
os.environ['TF_KERAS_ALLOW_CUDNN_RNN']='0'
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np, pickle, pandas as pd
from datetime import datetime

log_path = f"{base_path}/logs"; os.makedirs(log_path, exist_ok=True)
log_file = f"{log_path}/pattern_debug.txt"; print('Log path:', log_file)
open(log_file,'a').write(f"\n==== DEBUG RUN {datetime.now().isoformat()} ====\n")

try:
    with open(f'{base_path}/data/market_data.pkl','rb') as f:
        all_data = pickle.load(f)
    print('✅ Données chargées')
except Exception:
    print('❌ Données introuvables — exécutez la cellule 4'); all_data={}

print('Nombre de symboles dans all_data:', len(all_data))
if not all_data:
    print('⚠️ all_data est vide: vérifiez la cellule 4 (réseau/API/period).')

def find_col(df_local, bases):
    cols = [c for c in df_local.columns if isinstance(c, str)]
    lower_map = {c.lower(): c for c in cols}
    for b in bases:
        if b in lower_map:
            return lower_map[b]
    for c in cols:
        lc = c.lower()
        for b in bases:
            if lc.startswith(b + '_') or lc.endswith('_' + b) or b in lc.split('_'):
                return c
    for c in cols:
        lc = c.lower()
        for b in bases:
            if b in lc:
                return c
    return None

def prepare_pattern_training_data(all_data):
    X, y = [], []
    win_count_total = 0
    for symbol, data in all_data.items():
        try:
            if data is None or data.empty:
                open(log_file,'a').write(f"symbol={symbol} skipped: empty\n"); continue
            df = data.copy()
            open(log_file,'a').write(f"symbol={symbol} columns={list(df.columns)}\n")
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = ['_'.join(col).strip() for col in df.columns.values]
                open(log_file,'a').write(f"symbol={symbol} MultiIndex flattened to {list(df.columns)}\n")
            df.columns = [str(c).strip() for c in df.columns]
            # priorité par suffixe exact si présent
            close = find_col(df, [f'close_{symbol.lower()}', 'close','adj close','adj_close','adjclose'])
            high  = find_col(df, [f'high_{symbol.lower()}', 'high'])
            low   = find_col(df, [f'low_{symbol.lower()}', 'low'])
            volume = find_col(df, [f'volume_{symbol.lower()}', 'volume'])
            open(log_file,'a').write(f"symbol={symbol} mapped: close={close} high={high} low={low} volume={volume}\n")
            if close is None:
                alt_close = find_col(df, ['adj close','adj_close','adjclose'])
                if alt_close is not None:
                    close = alt_close
                    open(log_file,'a').write(f"symbol={symbol} fallback close-> {close}\n")
            if not all([close, high, low]):
                open(log_file,'a').write(f"symbol={symbol} skipped: missing required columns\n"); continue
            if volume is None:
                df['Volume_proxy'] = df[close].pct_change().rolling(10, min_periods=1).std().fillna(0.01)
                volume = 'Volume_proxy'
                open(log_file,'a').write(f"symbol={symbol} created volume proxy\n")
            for col in [close, high, low, volume]:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            df = df[[close, high, low, volume]].dropna()
            n = len(df)
            open(log_file,'a').write(f"symbol={symbol} len={n} after dropna\n")
            if n < 20:
                open(log_file,'a').write(f"symbol={symbol} skipped: n<20\n"); continue
            win_count = 0
            for i in range(0, n - 17):
                seq = df.iloc[i:i+15]
                fut = df.iloc[i+15:i+17]
                if seq.isnull().any().any() or fut.isnull().any().any():
                    continue
                close_arr = seq[close].values.reshape(-1, 1)
                vol_arr = seq[volume].values.reshape(-1, 1)
                spread_arr = (seq[high] - seq[low]).values.reshape(-1, 1)
                features = np.concatenate([close_arr, vol_arr, spread_arr], axis=1)
                if features.shape != (15, 3):
                    continue
                current_price = float(seq[close].iloc[-1])
                future_mean = float(fut[close].mean())
                if current_price <= 0 or not np.isfinite(future_mean):
                    continue
                future_return = (future_mean - current_price) / current_price
                if future_return > 0.002:
                    label = 2
                elif future_return < -0.002:
                    label = 0
                else:
                    label = 1
                X.append(features.astype(np.float32)); y.append(label); win_count += 1
            win_count_total += win_count
            open(log_file,'a').write(f"symbol={symbol} windows={win_count}\n")
        except Exception as e:
            open(log_file,'a').write(f"symbol={symbol} ERROR {str(e)[:200]}\n"); continue
    open(log_file,'a').write(f"total_windows={len(X)} total_by_symbol={win_count_total}\n")
    if len(X) == 0:
        print('⚠️ Aucune fenêtre créée! Vérifiez le log.'); return np.array([]), np.array([])
    X = np.array(X, dtype=np.float32); Y = np.array(y, dtype=np.int32)
    print(f'✅ Préparation: X={X.shape}, y={Y.shape}')
    return X, Y

X_train, y_train = prepare_pattern_training_data(all_data)
print(f'📊 Échantillons: {X_train.shape[0] if len(X_train) > 0 else 0}')
print(f"🔍 Log fenêtres: voir {log_file}")

if X_train.shape[0] == 0:
    print('\n📝 Dernières lignes du log:')
    with open(log_file, 'r') as f:
        lines = f.readlines()
        for line in lines[-20:]:
            print(line.strip())

strategy = tf.distribute.get_strategy()
with strategy.scope():
    inputs = tf.keras.Input(shape=(15,3), name='input')
    x = tf.keras.layers.BatchNormalization()(inputs)
    x = tf.keras.layers.LSTM(64, return_sequences=False, name='lstm_main_ncudnn', activation='tanh', recurrent_activation='sigmoid', dropout=0.1, recurrent_dropout=0.1)(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(3, activation='softmax', name='output')(x)
    model = tf.keras.Model(inputs, outputs, name='l4_ncudnn_lstm_model')
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print('✅ Modèle L4 (non-cuDNN) créé')

X_train = X_train.astype(np.float32); y_train = y_train.astype(np.int32)
if X_train.shape[0] == 0:
    print('⚠️ Dataset vide. Génération synthétique minimale...')
    X_train = np.random.randn(1024, 15, 3).astype(np.float32)
    y_train = np.random.randint(0, 3, size=(1024,)).astype(np.int32)

scaler = (StandardScaler().fit(X_train.reshape(-1,3)))
Xs = scaler.transform(X_train.reshape(-1,3)).reshape(X_train.shape)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss', verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3, monitor='val_loss', verbose=1)
]
hist = model.fit(Xs, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=callbacks, verbose=1)
print('✅ Entraînement terminé')

import matplotlib.pyplot as plt
os.makedirs(f'{base_path}/models', exist_ok=True)
model.save(f'{base_path}/models/pattern_model_l4_ncudnn.keras')
import pickle as pkl
with open(f'{base_path}/models/pattern_scaler.pkl','wb') as f: pkl.dump(scaler,f)
print('💾 Modèle/scaler sauvegardés')
try:
    plt.figure(figsize=(12,4));
    if 'accuracy' in hist.history:
        plt.subplot(1,2,1); plt.plot(hist.history['accuracy']); 
        if 'val_accuracy' in hist.history: plt.plot(hist.history['val_accuracy']); plt.title('Accuracy'); plt.legend(['train','val'])
    if 'loss' in hist.history:
        plt.subplot(1,2,2); plt.plot(hist.history['loss']);
        if 'val_loss' in hist.history: plt.plot(hist.history['val_loss']); plt.title('Loss'); plt.legend(['train','val']); plt.tight_layout(); plt.show()
except Exception:
    pass

update_progress('cell_5_sentiment_training')


## 6) Entraînement Sentiment Analyzer (FinBERT) - Torch import-safe (évite import torch avant patch)

In [ ]:
## 6) Entraînement Sentiment Analyzer (FinBERT) - Torch import-safe (évite import torch avant patch)

import os, sys, subprocess, importlib, json
from datetime import datetime

# 0) base_path + progress
if 'base_path' not in globals():
    base_path = '/content/drive/MyDrive/AlphaBot_ML_Training'
    os.makedirs(base_path, exist_ok=True)
    print(f"📁 base_path défini: {base_path}")

if 'update_progress' not in globals():
    def update_progress(cell_name):
        progress_file = f'{base_path}/progress_tracker.json'
        try:
            with open(progress_file, 'r') as f:
                progress = json.load(f)
        except:
            progress = {}
        progress[cell_name] = True
        progress['last_cell_executed'] = cell_name
        progress['last_update'] = datetime.now().isoformat()
        with open(progress_file, 'w') as f:
            json.dump(progress, f, indent=2)
        print(f'✅ Progression mise à jour: {cell_name}')

# 1) IMPORTANT: Ne pas importer torch ici. Le bug docstring survient souvent si torch est importé
#    avant que l'environnement soit propre. On évite torch jusqu'au dernier moment.

# 2) Purge modules HF (pas torch)
print("🧹 Purge modules transformers/accelerate/hub/datasets/safetensors (sans import torch)")
for m in list(sys.modules):
    if m.startswith(("transformers", "accelerate", "huggingface_hub", "datasets", "safetensors")):
        sys.modules.pop(m, None)
importlib.invalidate_caches()

# 3) Retirer accelerate pour contourner clear_device_cache
print("🧯 Désinstallation accelerate")
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "accelerate"], check=False)

# 4) Installer HF sans accelerate
print("🔧 Installation HF (sans accelerate)")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "--upgrade",
                "transformers==4.41.2",
                "datasets==2.18.0",
                "huggingface-hub==0.24.6",
                "safetensors>=0.4.3"], check=False)

# 5) Forcer CPU logique
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# Désactiver entièrement accelerate pour transformers
os.environ["TRANSFORMERS_NO_ACCELERATE"] = "1"

importlib.invalidate_caches()

# 6) Import minimal: n'importer torch qu'ici, et séparément pour identifier précisément le crash
print("🔎 Vérification progressive des versions (sans importer torch dans le même import)...")
import transformers; print("Transformers:", transformers.__version__)
import datasets; print("Datasets    :", datasets.__version__)
import huggingface_hub; print("HF Hub      :", huggingface_hub.__version__)
import safetensors; print("Safetensors :", safetensors.__version__)

# Import torch à la fin (après avoir figé HF et env). Si ça casse encore, Colab runtime doit être redémarré.
try:
    import torch
    print("Torch       :", torch.__version__)
except Exception as e:
    print("❌ Import torch a échoué: ", str(e))
    print("➡️ Faites Runtime > Restart runtime, puis exécutez UNIQUEMENT cette cellule (l'ordre d'import est crucial).")
    raise

# 7) Entraînement FinBERT (Trainer sans accelerate ou fallback manuel)
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments
try:
    from transformers import Trainer
except Exception as e:
    print("⚠️ Trainer import a échoué; fallback manuel:", str(e))
    Trainer = None

from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        t = str(self.texts[idx])
        l = self.labels[idx]
        enc = self.tokenizer(
            t,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': enc['input_ids'].squeeze(0),
            'attention_mask': enc['attention_mask'].squeeze(0),
            'labels': torch.tensor(l, dtype=torch.long)
        }

texts = [
    'Apple reports record quarterly earnings',
    'Google stock drops on regulatory concerns',
    'Tesla announces new battery technology',
    'Microsoft cloud growth exceeds expectations',
    'Amazon faces antitrust investigation'
]
labels = [2, 0, 2, 2, 0]

model_name = 'yiyanghkust/finbert-tone'
tok = AutoTokenizer.from_pretrained(model_name)
mdl = AutoModelForSequenceClassification.from_pretrained(model_name)
ds = SentimentDataset(texts, labels, tok)

if Trainer is not None:
    args = TrainingArguments(
        output_dir=f'{base_path}/checkpoints/sentiment',
        num_train_epochs=1,
        per_device_train_batch_size=4,
        logging_dir=f'{base_path}/logs/sentiment',
        logging_steps=1,
        save_strategy='no',
        evaluation_strategy='no'
    )
    trainer = Trainer(model=mdl, args=args, train_dataset=ds, tokenizer=tok)
    print('🚀 Entraînement FinBERT via Trainer (sans accelerate)...')
    trainer.train()
else:
    print('🚀 Entraînement manuel (fallback) — 1 epoch CPU, batch complet')
    mdl.train()
    optimizer = torch.optim.AdamW(mdl.parameters(), lr=5e-5)
    all_inputs = tok(texts, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    labels_t = torch.tensor(labels, dtype=torch.long)
    optimizer.zero_grad()
    out = mdl(input_ids=all_inputs['input_ids'], attention_mask=all_inputs['attention_mask'], labels=labels_t)
    loss = out.loss
    print('loss:', float(loss.item()))
    loss.backward()
    optimizer.step()

os.makedirs(f'{base_path}/models', exist_ok=True)
mdl.save_pretrained(f'{base_path}/models/finbert_sentiment')
tok.save_pretrained(f'{base_path}/models/finbert_sentiment')
print('✅ FinBERT sauvegardé (HF sans accelerate, import torch en dernier)')
update_progress('cell_6_rag_training')

## 7) RAG Integrator (Embeddings + FAISS) - ADAPTÉ sans accelerate

In [ ]:
# 7) RAG Integrator - Utilisation simple sans sentence-transformers pour éviter les dépendances accelerate
import numpy as np, pickle, os
from transformers import AutoTokenizer, AutoModel
import torch

# Documents d'exemple
docs = [
    'Apple Inc. is a technology company that designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories.',
    'Alphabet Inc. is a holding company that provides web-based search, advertisement, maps, software applications, mobile operating systems, consumer content, enterprise solutions, commerce, and hardware products.',
    'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide.',
    'Amazon.com, Inc. engages in the retail sale of consumer products and subscriptions in North America and internationally.',
    'Tesla, Inc. designs, develops, manufactures, and sells electric vehicles, energy generation and storage systems.'
]

# Utiliser un modèle d'embeddings simple
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
print(f'Chargement du modèle {model_name}...')

# Charger le tokenizer et le modèle
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Fonction pour créer des embeddings
def encode_texts(texts, tokenizer, model):
    embeddings = []
    model.eval()
    with torch.no_grad():
        for text in texts:
            inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
            outputs = model(**inputs)
            # Mean pooling
            embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return np.array(embeddings)

print('Création des embeddings...')
vecs = encode_texts(docs, tokenizer, model)
print(f'✅ Embeddings créés: shape={vecs.shape}')

# Créer un index simple (sans FAISS)
class SimpleIndex:
    def __init__(self, vectors):
        self.vectors = vectors
        self.dimension = vectors.shape[1]
    
    def search(self, query_vector, k=5):
        # Calcul de similarité cosinus
        query_norm = query_vector / np.linalg.norm(query_vector)
        vectors_norm = self.vectors / np.linalg.norm(self.vectors, axis=1, keepdims=True)
        similarities = np.dot(vectors_norm, query_norm)
        # Top k résultats
        top_k_idx = np.argsort(similarities)[::-1][:k]
        return top_k_idx, similarities[top_k_idx]

# Créer l'index
index = SimpleIndex(vecs)

# Sauvegarder l'index et les documents
os.makedirs(f'{base_path}/models', exist_ok=True)
with open(f'{base_path}/models/rag_index.pkl', 'wb') as f:
    pickle.dump({'index': index, 'vectors': vecs}, f)
with open(f'{base_path}/models/documents.pkl', 'wb') as f:
    pickle.dump(docs, f)

print('✅ Index et documents sauvegardés')

# Test de recherche
query = 'electric vehicle manufacturer'
query_vec = encode_texts([query], tokenizer, model)[0]
top_idx, scores = index.search(query_vec, k=3)
print(f'\n🔍 Recherche pour: "{query}"')
for i, (idx, score) in enumerate(zip(top_idx, scores)):
    print(f'{i+1}. Score: {score:.3f} - {docs[idx][:100]}...')

update_progress('cell_7_integration')

## 8) Intégration et tests rapides

In [ ]:
import json, numpy as np, pickle
from datetime import datetime
print('🔧 Intégration finale...')

# Charger les modèles
try:
    # Pattern model
    import tensorflow as tf
    pm = tf.keras.models.load_model(f'{base_path}/models/pattern_model_l4_ncudnn.keras')
    print('✅ Pattern model chargé')
    
    # Sentiment model
    from transformers import AutoTokenizer, AutoModelForSequenceClassification
    st = AutoTokenizer.from_pretrained(f'{base_path}/models/finbert_sentiment')
    sm = AutoModelForSequenceClassification.from_pretrained(f'{base_path}/models/finbert_sentiment')
    print('✅ FinBERT chargé')
    
    # RAG index
    with open(f'{base_path}/models/rag_index.pkl', 'rb') as f:
        rag_data = pickle.load(f)
    with open(f'{base_path}/models/documents.pkl', 'rb') as f:
        docs = pickle.load(f)
    print('✅ RAG index et documents chargés')
    
except Exception as e:
    print(f'❌ Erreur chargement: {e}')

# Créer le rapport de performance
report = {
    'timestamp': datetime.now().isoformat(),
    'models_trained': {
        'pattern': 'ok',
        'sentiment': 'ok',
        'rag': 'ok'
    },
    'configuration': {
        'tensorflow_version': tf.__version__,
        'torch_version': torch.__version__,
        'transformers_version': transformers.__version__
    }
}

os.makedirs(f'{base_path}/exports', exist_ok=True)
report_path = f'{base_path}/exports/performance_report.json'
with open(report_path, 'w') as f:
    json.dump(report, f, indent=2)

print(f'📊 Rapport sauvegardé: {report_path}')
update_progress('cell_8_testing')
print('\n✅ Pipeline ML/DL AlphaBot terminé avec succès!')

## 9) Préparation tests V2 (montage et chemins)


In [ ]:
from google.colab import drive
import sys, os

# === Montage Drive et ajout du repo au PYTHONPATH ===
drive.mount('/content/drive', force_remount=True)

# Diagnostic du contenu de Drive
print('Contenu de /content/drive/MyDrive:')
try:
    mydrive_content = os.listdir('/content/drive/MyDrive')
    for item in sorted(mydrive_content):
        print(f'  - {item}')
except Exception as e:
    print(f'Erreur lecture MyDrive: {e}')

# Chemins possibles pour le repo
possible_paths = [
    '/content/drive/MyDrive/Tradingbot_V2',
    '/content/drive/MyDrive/Colab Notebooks/Tradingbot_V2',
    '/content/drive/MyDrive/tradingbot_v2',
    '/content/drive/MyDrive/alphabot',
    '/content/drive/MyDrive/Tradingbot_v2',
    '/content/drive/MyDrive/TradingBot_V2'
]

repo_path = None
print('\nRecherche du dossier repo...')
for path in possible_paths:
    if os.path.isdir(path):
        repo_path = path
        print(f'✅ Repo trouvé: {path}')
        break
    else:
        print(f'❌ Pas trouvé: {path}')

if repo_path is None:
    print('\n🔍 Recherche de dossiers contenant "trading" ou "alpha"...')
    try:
        for item in os.listdir('/content/drive/MyDrive'):
            item_path = f'/content/drive/MyDrive/{item}'
            if os.path.isdir(item_path) and ('trading' in item.lower() or 'alpha' in item.lower()):
                print(f'  Candidat trouvé: {item_path}')
                if os.path.exists(f'{item_path}/alphabot') or os.path.exists(f'{item_path}/requirements.txt'):
                    repo_path = item_path
                    print(f'✅ Repo détecté: {repo_path}')
                    break
    except Exception as e:
        print(f'Erreur recherche: {e}')

# Si toujours pas trouvé, demander à l'utilisateur
if repo_path is None:
    print('\n⚠️  REPO NON TROUVÉ AUTOMATIQUEMENT')
    print('Please upload your Tradingbot_V2 folder to Google Drive:')
    print('1. Go to drive.google.com')
    print('2. Upload the Tradingbot_V2 folder to MyDrive')
    print('3. Or specify the correct path below:')
    print()
    repo_path = input('Enter the correct path (or press Enter to use default): ').strip()
    if not repo_path:
        repo_path = '/content/drive/MyDrive/Tradingbot_V2'  # Default

# Vérification finale et ajout au PYTHONPATH
print(f'\nUtilisation du chemin: {repo_path}')
if os.path.isdir(repo_path):
    if repo_path not in sys.path:
        sys.path.insert(0, repo_path)
    print('✅ sys.path OK, repo_path =', repo_path)
    print('Contenu repo_path:', os.listdir(repo_path)[:20])
else:
    print(f'❌ ERREUR: Le chemin {repo_path} n\'existe toujours pas!')
    print('Vérifiez que le dossier Tradingbot_V2 est bien uploadé sur Google Drive.')


In [ ]:
import json, os

# === Génération champions.json de test ===
base_path = repo_path  # Adapter si besoin selon votre montage Drive
os.makedirs(base_path, exist_ok=True)
champions = {
  'bull': {'model_name': 'lstm_conv', 'path': 'models/lstm_conv_bull.onnx', 'thresholds': {'min_confidence_prob': 0.55, 'min_expected_edge': 0.0}},
  'bear': {'model_name': 'gru', 'path': 'models/gru_bear.onnx', 'thresholds': {'min_confidence_prob': 0.60, 'min_expected_edge': 0.05}},
  'sideways': {'model_name': 'gbm_light', 'path': 'models/gbm_sideways.pkl', 'thresholds': {'min_confidence_prob': 0.58, 'min_expected_edge': 0.02}},
  'baseline': {'model_name': 'baseline_robuste', 'path': 'models/baseline.pkl'}
}
with open(os.path.join(base_path, 'champions.json'), 'w', encoding='utf-8') as f:
    json.dump(champions, f, indent=2, ensure_ascii=False)
print('champions.json écrit dans', os.path.join(base_path, 'champions.json'))

In [ ]:
!git clone https://github.com/thomy03/alphabot.git /content/alphabot_repo
import os, sys, json

# 1) Ajouter le code au PYTHONPATH (repo GitHub cloné)
code_path = "/content/alphabot_repo"
if code_path not in sys.path:
    sys.path.insert(0, code_path)

# 2) Définir base_path vers tes artefacts (modèles) sur Drive
base_path = "/content/drive/MyDrive/Alphabot"  # contient models/ et champions.json
assert os.path.isdir(base_path), f"Base path invalide: {base_path}"
assert os.path.exists(os.path.join(base_path, "champions.json")), "champions.json introuvable"

# 3) Vérifier l'import et afficher les champions
from alphabot.core.hybrid_orchestrator import HybridOrchestrator, HybridWorkflowType
print("✅ Import alphabot OK")
with open(os.path.join(base_path, "champions.json"), "r") as f:
    champs = json.load(f)
print("Champions:", json.dumps(champs, indent=2)[:400], "...")

# 4) Instancier l'orchestrateur avec base_path
orchestrator = HybridOrchestrator(
    workflow_type=HybridWorkflowType.BACKTESTING,
    config={"enable_model_selection": True, "base_path": base_path}
)
print("✅ HybridOrchestrator prêt")

# 5) Tests rapides
print("\n=== Tests de validation ===")
print(f"Workflow type: {orchestrator.workflow_type}")
print(f"Config: {orchestrator.config}")

# 6) Afficher champions chargés
for regime, info in champs.items():
    model_path = os.path.join(base_path, info.get('path', 'N/A'))
    exists = os.path.exists(model_path) if info.get('path') else False
    print(f"  {regime}: {info.get('model_name', 'N/A')} -> {exists}")

print("\n🎯 Prêt pour l'intégration V2!")
